![Pytorch](images/pytorch_logo.png)
# Regression in Pytorch with all the bells and whistles
Pytorch lets us skip a bunch of math, but that's only part of why it's heavily used in Deep Learning. It is after all a Deep Learning framework, not a maths framework!

Let's redo our simple Linear Regression, but this time we use all the bells and whistles!

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Set seed
seed = 42
torch.manual_seed(seed);

## Load data

We are using the built-in sklearn dataset Boston House Prices.

Our goal is to predict the median price of a home in a given town from a number of features, such as Crime Rate, Property Tax Rate, amount of Industry etc.

It's generally a good idea to scale our data, so we use Sklearn's MinMax scaler to scale our values between 0 and 1

In [ ]:
# Load our dataset
boston = load_boston()
train_x, test_x, train_y, test_y = train_test_split(boston.data, boston.target, random_state=seed)
scaler = MinMaxScaler()


train_x = torch.tensor(scaler.fit_transform(train_x), dtype=torch.float)
test_x = torch.tensor(scaler.transform(test_x), dtype=torch.float)
train_y = torch.tensor(train_y, dtype=torch.float).view(-1, 1)
test_y = torch.tensor(test_y, dtype=torch.float).view(-1, 1)

## Setup parameters

We have some hyperparameters to set, as well as some numbers we need to know upfront.

`layer_size` --> We need to know how many input variables there are, so we can create an equivalent number of weights

`lr` --> Aka learning rate.
When we take a step in our gradient descent, we multiply by this factor, so we don't take too big or too large a step. 

`epochs` --> How many times should we keep stepping?

In [ ]:
# Set some parameters
layer_size = train_x.shape[1]
lr = 0.05
epochs = 700

# Defining the Model
We get two new toys here:

- `nn.Module`

Now we can define our Model simply by inheriting from `nn.module`. We define a forward function to use for the forward pass.

- `nn.Linear`

We can now use these built-in layers such as `nn.Linear` which handle all the parameters for us

In [ ]:
# Define model
class Regression(nn.Module):
    def __init__(self, layer_size):
        super().__init__()
        self.layer_1 = nn.Linear(layer_size, 1) # equivalent of wx + b
        
    def forward(self, x):
        return self.layer_1(x) 

# Define loss function and optimizer
Pytorch predefines a number of loss functions for me, so I can just use those directly.
Another new features is the SGD optimizer. Until now we have been updating the weights "by hand" in a fairly naive fashion. There are many ways of updating our weights and Pytorch provides implementations for most of these. SGD is the closest to what we've been doing so far, so we use that to handle our weight updates.

In [ ]:
model = Regression(layer_size) # Defines our parameters for us
loss_func = nn.MSELoss() # Define loss func
opt = optim.SGD(model.parameters(), lr=lr) # Tell the optimizer what parameters to keep track of

In [ ]:
# Training loop
for epoch in range(epochs):
    # Forward pass
    model.train() # Put the model into train mode
    pred = model(train_x) # Our model acts like a function!
    loss = loss_func(pred, train_y)
    
    # Backpropagation
    loss.backward() # The magic bit
    opt.step() # A new magic bit
    opt.zero_grad() # Gotta reset the gradients to zero, so they don't accumulate
    
    # Validate model
    model.eval() # Put the model into evaluation mode
    with torch.no_grad():
        val_pred = model(test_x)
        val_loss = loss_func(val_pred, test_y) # Calculate validation loss
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} Training Loss: {loss.item()} Test Loss: {val_loss.item()}")